In [31]:
import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

# 파일들이 저장된 폴더 경로를 지정합니다.
folder_path = './data_240616/'  # 여기에 실제 폴더 경로를 넣으세요

# 새로운 엑셀 파일에 데이터를 모으기 위해 빈 데이터프레임을 생성합니다.
combined_df = pd.DataFrame()

# 저장할 파일의 이름을 지정합니다.
output_file = 'combined_sheet2.xlsx'
output_file_path = os.path.join(folder_path, output_file)

# 저장할 파일이 이미 존재하면 삭제합니다.
if os.path.exists(output_file_path):
    os.remove(output_file_path)
    print(f"기존 파일 {output_file}을 삭제했습니다.")

# 폴더 내의 모든 엑셀 파일을 순회합니다.
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx") and filename != output_file:
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        try:
            # 각 파일의 sheet2 데이터를 읽어옵니다.
            df = pd.read_excel(file_path, sheet_name='sheet2')
            # 데이터를 모으기 위한 데이터프레임에 추가합니다.
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except Exception as e:
            print(f"파일 {filename}에서 sheet2를 읽어오는 데 문제가 발생했습니다: {e}")

# 평균(면적/3.3) 열을 기준으로 내림차순 정렬합니다.
if '평균(면적/3.3)' in combined_df.columns:
    combined_df.sort_values(by='평균(면적/3.3)', ascending=False, inplace=True)
else:
    print("데이터프레임에 '평균(면적/3.3)' 열이 없습니다.")

# 평균(면적/3.3)이 50000을 넘는 행을 삭제합니다.
combined_df = combined_df[combined_df['평균(면적/3.3)'] <= 50000]

# 평균(면적/3.3)과 count 열 값을 곱하여 총액열을 추가합니다.
combined_df['총액'] = combined_df['평균(면적/3.3)'] * combined_df['count']

# 데이터를 모은 새로운 엑셀 파일을 저장합니다.
combined_df.to_excel(output_file_path, index=False, sheet_name='data')

# openpyxl을 사용하여 특정 열의 셀 크기를 조정합니다.
wb = load_workbook(output_file_path)
ws = wb['data']

# '법정동명'과 '아파트명' 열의 셀 크기를 조정합니다.
specific_column_widths = {'법정동명': 30, '아파트명': 30}
default_width = 12

# 열 너비 설정
for col in ws.columns:
    col_letter = get_column_letter(col[0].column)
    column_name = col[0].value
    if column_name in specific_column_widths:
        ws.column_dimensions[col_letter].width = specific_column_widths[column_name]
    else:
        ws.column_dimensions[col_letter].width = default_width

# 숫자 열에 세 자리 단위로 콤마 추가
for col in ws.iter_cols(min_row=2, min_col=1, max_col=ws.max_column):
    column_name = ws.cell(row=1, column=col[0].column).value
    if pd.api.types.is_numeric_dtype(combined_df[column_name]):
        for cell in col:
            cell.number_format = '#,##0'



# 데이터프레임을 pivot하여 summary 시트에 저장합니다.
pivot_df = combined_df.pivot_table(index='법정동명',
                                   values=['count', '총액'],
                                   aggfunc={'count': 'sum', '총액': 'sum'})

# 평당가격 열 추가
pivot_df['평당가격'] = pivot_df['총액'] / pivot_df['count']

# summary 시트에 pivot 테이블 저장
summary_ws = wb.create_sheet(title='summary')
summary_ws.append(['지역1', '평균 평당가격'])

# 법정동명을 summary 시트에 저장합니다.
for idx, row in pivot_df.sort_values(by='평당가격', ascending=False).iterrows():
    summary_ws.append([idx, '{:,.0f}'.format(row['평당가격'])])

# summary 시트의 열 너비 설정
summary_ws.column_dimensions['A'].width = 30
summary_ws.column_dimensions['B'].width = 20

# pivot 테이블 생성 및 rank 시트에 저장
pivot_df2 = pivot_df.groupby(pivot_df.index.str.split(' ').str[-2])['평당가격'].mean().sort_values(ascending=False)
rank_ws = wb.create_sheet(title='rank')
rank_ws.append(['지역1', '평균 평당가격'])

# rank 시트에 평균 평당가격을 저장합니다.
for index, avg_price in pivot_df2.items():
    formatted_avg_price = '{:,.0f}'.format(avg_price)
    rank_ws.append([index, formatted_avg_price])

# rank 시트의 열 너비 설정
rank_ws.column_dimensions['A'].width = 15
rank_ws.column_dimensions['B'].width = 20

# 엑셀 파일 저장
wb.save(output_file_path)

print(f"모든 파일의 sheet2 데이터를 {output_file} 파일에 저장하고, summary 및 rank 시트에 pivot 데이터를 저장했습니다.")





기존 파일 combined_sheet2.xlsx을 삭제했습니다.
./data_240616/강남구_properties_20240616_131915.xlsx
./data_240616/강동구_properties_20240616_125544.xlsx
./data_240616/강릉시_properties_20240617_140951.xlsx
./data_240616/강북구_properties_20240616_165000.xlsx
./data_240616/경산시_properties_20240617_132818.xlsx
./data_240616/광주시경기_properties_20240616_232130.xlsx
./data_240616/광진구_properties_20240617_124145.xlsx
./data_240616/괴산군_properties_20240617_123637.xlsx
./data_240616/구리시_properties_20240616_171340.xlsx
./data_240616/기흥구_properties_20240617_145648.xlsx
./data_240616/남양주시_properties_20240617_090219.xlsx
./data_240616/노원구_properties_20240616_161549.xlsx
./data_240616/달서구_properties_20240617_092844.xlsx
./data_240616/도봉구_properties_20240616_162758.xlsx
./data_240616/동대문구_properties_20240617_130701.xlsx
./data_240616/동대문구_properties_20240617_135446.xlsx
./data_240616/동작구_properties_20240616_143406.xlsx
./data_240616/마포구_properties_20240616_150000.xlsx
./data_240616/분당구_properties_20240617_105749.xlsx
./data_24